## Plotly interactive spherical Voronoi diagram ##

Starting with version 0.18.0, `scipy` provides the class [`scipy.spatial.SphericalVoronoi`](http://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.SphericalVoronoi.html), that defines a spherical Voronoi diagram associated to a set of points on a sphere. 

In the example posted at the above link, the diagram is visualized as a `mpl_toolkits.mplot3d.art3d.Poly3DCollection`. The collection of polygons has only the vertices on the sphere and  a plot of the corresponding collection is  an approximation of the spherical Voronoi diagram.
                                             
In this Jupyter Notebook we project the boundaries of the polygons onto the sphere, in order to get the true boundaries of the spherical Voronoi regions. The spherical Voronoi diagram is visualized as an interactive Plotly plot.

In [2]:
import scipy
scipy. __version__

'0.18.0'

In [3]:
from scipy.spatial import SphericalVoronoi
from scipy.interpolate import splev,  splprep
import numpy as np

In [4]:
import plotly.plotly as py
from plotly.graph_objs import *

Discretize the unit sphere, S(center=O, radius=1):

In [5]:
theta=np.linspace(0, 2*np.pi, 200)
phi=np.linspace(0, np.pi, 100)
theta, phi=np.meshgrid(theta, phi)
x=np.cos(theta)*np.sin(phi)
y=np.sin(theta)*np.sin(phi)
z=np.cos(phi)

Generate N points uniformly distributed on the unit sphere:

In [18]:
N=80
points=np.random.randn(N,3)
points=points/np.linalg.norm(points, axis=1)[:, None]

Define an instance of the `SphericalVoronoi` class:

In [19]:
center = np.zeros(3)
radius = 1.0
sv = SphericalVoronoi(points, radius, center)
sv.sort_vertices_of_regions()# sort vertices

  - `sv.vertices` is the numpy array of all Voronoi vertices (3d points on the unit sphere).
  - `sv.regions` is a list of lists. An inner list stores the indices of vertices associated to a region:

In [20]:
sv.regions[0]

[20, 48, 50, 49]

Define a custom Plotly colorscale to plot the sphere:

In [21]:
pl_col=[[0.0, 'rgb(230,230,230)'], [1.0, 'rgb(230,230,230)']]

The sphere is defined as a Plotly `Surface`:

In [22]:
sphere=Surface(x=x, y=y, z=z, colorscale=pl_col, showscale=False, name='sphere')

Plot the data points and the boundaries of the Voronoi regions on a sphere of radius, R,  slightly greater than 1,
in order to avoid hiding them by the sphere surface:

In [23]:
R=1.005
data_pts=Scatter3d(x=R*points[:,0], y=R*points[:,1], z=R*points[:,2], name='points',
                   mode='markers', marker=dict(color='black', size=3))

For each pair of consecutive points, $P_k, P_{k+1}$, defining a Voronoi region, compute 5 points on the segment having these points as ends.
The five points are projected (via a central projection of center O) onto the sphere of radius $R$, and the  corresponding points  are spline interpolated to get a spherical arc:

In [24]:
t=np.array([0, 0.25, 0.5, 0.75, 1.0])# five parameters for convex combination of points
p=(1-t)[:, None]# equiv to [:, np.newaxis]
q=t[:, None]
#Xe, Ye, Ze  are the lists of point coordinates to be plotted as spherical arcs
Xe=[]
Ye=[]
Ze=[]
for region in sv.regions:
    P=sv.vertices[region]#P is an array whose rows are the vertices of the Voronoi points on the sphere
    L=P.shape[0]
    for k in range(L):
        B=np.array([P[k,:]]*5)
        C=np.array([P[(k+1)%L, :]]*5)
        A=B*p+C*q#A is an array of 5 points on the segment of ends P[k,:], P[(k+1)%L, :]
       
        A=R*A/np.linalg.norm(A, axis=1)[:, None]#central projection of the points in A onto the sphere of rad. R 
        tck,u=splprep([A[:,0],A[:,1],A[:,2]],s=0)
        xi,yi, zi= splev(np.linspace(0,1,20),tck)#spline interpolation of the five points on sphere
        Xe+=xi.tolist()
        Ye+=yi.tolist()
        Ze+=zi.tolist()
    Xe+=[None]  #after processing a region insert None in each list to avoid 
    Ye+=[None]  #unwanted lines from one region to another
    Ze+=[None]
    


Define the Plotly Scatter3d object, consisting in all spherical arcs, computed above:

In [25]:
lines=Scatter3d(x=Xe, y=Ye, z=Ze,  name='spheric arc',  mode='lines',
                line=dict(width=2, color='rgb(10,10,10)'))    

Set the plot layout (with axes or not):

In [26]:
axis = dict(
            showbackground=True, 
            backgroundcolor='rgb(40,40,40)', #"rgb(230, 230,230)",
            gridcolor='rgb(255, 255, 255)',      
            zerolinecolor='rgb(255, 255, 255)',  
            )
noaxis=dict(showbackground=False,
            showgrid=False,
            showline=False,
            showticklabels=False,
            ticks='',
            title='',
            zeroline=False)

In [27]:
def plot_layout(ax=noaxis):

    return Layout(title='Spherical Voronoi Diagram',
                  font=dict(family='Balto', size=16),
                  width=700,
                  height=700,
                  showlegend=False,
                  scene=Scene(xaxis=XAxis(ax),
                              yaxis=YAxis(ax), 
                              zaxis=ZAxis(ax), 
                              aspectratio=dict(x=1,
                                               y=1,
                                               z=1
                                              ),
                                )
                    )
  


Data to be plotted:

In [28]:
data=Data([sphere, data_pts, lines])

A plot without axes:

In [29]:
fig1 = Figure(data=data, layout=plot_layout())
py.sign_in('empet', 'my_api_key')
py.iplot(fig1, filename='sph-voronoi-noaxes')

and with axes and background:

In [30]:
fig2 = Figure(data=data, layout=plot_layout(ax=axis))
py.iplot(fig2, filename='sph-voronoi-axes')

In [31]:
from IPython.core.display import HTML
def  css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()